Cleaner version of the notebook that I run on laptop to scape data faster

In [1]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os

In [2]:
#captcha alert settings
import winsound
duration = 5000  # milliseconds
freq = 440  # Hz

In [3]:
headers_list = [
# Firefox 77 Mac
{
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Firefox 77 Windows
{
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Chrome 83 Mac
{
"Connection": "keep-alive",
"DNT": "1",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "none",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
},
# Chrome 83 Windows 
{
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9"
}
]

In [4]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 5000
cities = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
          'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
          'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Boston',
          'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose',
          'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
          'Memphis', 'Orlando', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
          'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
          'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
          'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 
          'Pittsburgh', 'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 
          'Herndon','Reston', 'Roanoke', 'Boulder', 'Pasadena', 'Greensboro', 'Newark', 'Kansas+City', 'Abilene',
          "Baton Rouge", 'Bridgeport', 'Lubbock', 'Amarillo', 'Oklahoma+City', 'Clinton', 'Stillwater', 'Berkeley',
          'Palo+Alto', 'Cupertino', 'Buffalo', 'Dayton']

In [5]:
i = 0
count = 0
results = []

for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        file = 'data_by_cities/'+str(city)+'.csv'
        try: 
            df_more = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
            print('file successfully opened')
        except:
            print('file not found, creating new file')
            df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
            df_more.to_csv(file, sep='\t', encoding='utf-8')
        datasize = df_more.shape
        url = url_template.format(city, start)
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        html = r.get(url)
        #html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        #captcha detector
        if 'hCaptcha solve page' in soup.get_text():
            print('captcha detected')
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobTitle').text.replace('\n', '').replace('new', '')
            except:
                title = 'None'
            try:
                location = each.find('div', {'class':"companyLocation" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='companyName').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('div', {'class': "metadata salary-snippet-container"}).text.replace('\n', '')
            except:
                try:
                    salary = each.find('span', {'class':'estimated-salary'}).text.replace('\n', '')
                except:
                    salary = 'None'
            synopsis = each.find('div', {'class':'job-snippet'}).text.replace('\n', '')
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            #drop duplicates
            df_more = df_more[~df_more.duplicated()].reset_index(drop=True)
        #count the number of pages scraped and print, so I know how far the code is running
        count+= 1
        print(count, ' loops done, proceed to sleep, current city is ', city)
        #move to next city if we don't get new data. it's pointless to continue without getting new data.
        if df_more.shape == datasize:
            print(df_more.shape,' no new data, moving to next city')
            break       
        df_more.to_csv(file, sep='\t', encoding='utf-8')#write every page so i don't lose my data again
        sleep(random.randint(7,20))
        #I don't want too see long line of the process, clear_output will clear previous lines
        clear_output(wait=True)
        print(df_more.shape, ' data saved, last visited city is ', city, ', continuing....')
    print('moving to next city')
    sleep(random.randint(10,30))

(912, 5)  data saved, last visited city is  Reston , continuing....
file successfully opened
634  loops done, proceed to sleep, current city is  Reston
(912, 5)  no new data, moving to next city
moving to next city
file successfully opened
635  loops done, proceed to sleep, current city is  Tucson
(35, 5)  no new data, moving to next city
moving to next city


In [6]:
retval = os.getcwd()
print ("Current working directory %s" % retval)

Current working directory C:\Users\silve\OneDrive\Dokumente\Github\Coding-fun\Web Scraping Job Postings


In [7]:
import glob
os.chdir(".\data_by_cities")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['Abilene.csv',
 'Albuquerque.csv',
 'Alexandria.csv',
 'Amarillo.csv',
 'Anaheim.csv',
 'Anchorage.csv',
 'Arlington.csv',
 'Atlanta.csv',
 'Aurora.csv',
 'Austin.csv',
 'Bakersfield.csv',
 'Baltimore.csv',
 'Baton Rouge.csv',
 'Baton+Rouge.csv',
 'Berkeley.csv',
 'Boston.csv',
 'Boulder.csv',
 'Bridgeport.csv',
 'Buffalo.csv',
 'Charlotte.csv',
 'Charlottesville.csv',
 'Chesapeake.csv',
 'Chicago.csv',
 'Cincinnati.csv',
 'cleaned_data.csv',
 'Cleveland.csv',
 'Clinton.csv',
 'Colorado+Springs.csv',
 'Columbus.csv',
 'Corpus+Christi.csv',
 'Cupertino.csv',
 'Dallas.csv',
 'Dayton.csv',
 'Denver.csv',
 'Detroit.csv',
 'El+Paso.csv',
 'Fairfax.csv',
 'Fort+Worth.csv',
 'Fresno.csv',
 'Greensboro.csv',
 'Harrisonburg.csv',
 'Herndon.csv',
 'Honolulu.csv',
 'Houston.csv',
 'Indianapolis.csv',
 'Jacksonville.csv',
 'Kansas+City.csv',
 'Las+Vegas.csv',
 'Lexington.csv',
 'Long+Beach.csv',
 'Los+Angeles.csv',
 'Louisville.csv',
 'Lubbock.csv',
 'Memphis.csv',
 'Mesa.csv',
 'Miami.csv',
 'Mi

In [8]:
#combine all files in the list
combined_csv = pd.concat([ pd.read_csv(f, sep='\t', encoding='utf-8', index_col=0) for f in all_filenames ] , ignore_index=True)

In [9]:
combined_csv

,Title,Location,Company,Salary,Synopsis
0,Senior Data Scientist - Delivery Insights,Remote in Texas,Indeed,"$139,000 - $201,000 a year",Passion to answer Product/Engineering question...
1,Senior Data Analyst,Remote in Texas,Hire With Jarvis,"$120,000 - $170,000 a year",Expertise in SQL and data visualization.Experi...
2,Data Scientist - (2+ yrs of statistics & R pro...,Texas,#twiceasnice Recruiting,Estimated $96.7K - $122K a year,Salary: Commensurate with Experience + Bonus +...
3,Data Scientist - Entry to Expert Level Virtual...,United States,National Security Agency (NSA),"$74,682 - $176,300 a year",Passing this examination is a requirement in o...
4,"Director, Data Science- SMB Labs",Remote in United States,Indeed,"$214,000 - $310,000 a year","In this role, you will lead teams of machine l..."
...,...,...,...,...,...
19120,Senior Data Scientist at Rent-to-Own Startup,"Remote in Wichita, KS 67212",Recruiting From Scratch,"$120,000 - $190,000 a year",Who is Recruiting from Scratch: Recruiting fro...
19121,Senior Data Scientist/Machine Learning Engineer,"Remote in Wichita, KS 67212+5 locations",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...
19122,Data Plane Engineer,"Remote in Wichita, KS 67212",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...
19123,NLP Engineer at Speech to Text Start Up,"Remote in Wichita, KS 67212",Recruiting From Scratch,Estimated $115K - $145K a year,Who is Recruiting from Scratch: Recruiting fro...


In [10]:
combined_csv = combined_csv[~combined_csv.duplicated()].reset_index(drop=True)

In [11]:
combined_csv

,Title,Location,Company,Salary,Synopsis
0,Senior Data Scientist - Delivery Insights,Remote in Texas,Indeed,"$139,000 - $201,000 a year",Passion to answer Product/Engineering question...
1,Senior Data Analyst,Remote in Texas,Hire With Jarvis,"$120,000 - $170,000 a year",Expertise in SQL and data visualization.Experi...
2,Data Scientist - (2+ yrs of statistics & R pro...,Texas,#twiceasnice Recruiting,Estimated $96.7K - $122K a year,Salary: Commensurate with Experience + Bonus +...
3,Data Scientist - Entry to Expert Level Virtual...,United States,National Security Agency (NSA),"$74,682 - $176,300 a year",Passing this examination is a requirement in o...
4,"Director, Data Science- SMB Labs",Remote in United States,Indeed,"$214,000 - $310,000 a year","In this role, you will lead teams of machine l..."
...,...,...,...,...,...
9965,Senior Data Scientist at Rent-to-Own Startup,"Remote in Wichita, KS 67212",Recruiting From Scratch,"$120,000 - $190,000 a year",Who is Recruiting from Scratch: Recruiting fro...
9966,Senior Data Scientist/Machine Learning Engineer,"Remote in Wichita, KS 67212+5 locations",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...
9967,Data Plane Engineer,"Remote in Wichita, KS 67212",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...
9968,NLP Engineer at Speech to Text Start Up,"Remote in Wichita, KS 67212",Recruiting From Scratch,Estimated $115K - $145K a year,Who is Recruiting from Scratch: Recruiting fro...


In [12]:
rslt_df = combined_csv[combined_csv['Salary'] != 'None']
estimated = rslt_df.loc[rslt_df['Salary'].str.contains("Estimated")].index
rslt_df.drop(estimated, inplace=True)
rslt_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Title,Location,Company,Salary,Synopsis
0,Senior Data Scientist - Delivery Insights,Remote in Texas,Indeed,"$139,000 - $201,000 a year",Passion to answer Product/Engineering question...
1,Senior Data Analyst,Remote in Texas,Hire With Jarvis,"$120,000 - $170,000 a year",Expertise in SQL and data visualization.Experi...
3,Data Scientist - Entry to Expert Level Virtual...,United States,National Security Agency (NSA),"$74,682 - $176,300 a year",Passing this examination is a requirement in o...
4,"Director, Data Science- SMB Labs",Remote in United States,Indeed,"$214,000 - $310,000 a year","In this role, you will lead teams of machine l..."
5,Data Science Manager - Search Ranking and Pers...,Remote in United States,Indeed,"$149,000 - $217,000 a year","Ensure delivery of data science solutions, set..."
...,...,...,...,...,...
9925,Principal ML & NLP Engineer,"Remote in Clearwater, FL 33759",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...
9965,Senior Data Scientist at Rent-to-Own Startup,"Remote in Wichita, KS 67212",Recruiting From Scratch,"$120,000 - $190,000 a year",Who is Recruiting from Scratch: Recruiting fro...
9966,Senior Data Scientist/Machine Learning Engineer,"Remote in Wichita, KS 67212+5 locations",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...
9967,Data Plane Engineer,"Remote in Wichita, KS 67212",Recruiting From Scratch,"$140,000 - $200,000 a year",Who is Recruiting from Scratch: Recruiting fro...


In [13]:
rslt_df.to_csv('cleaned_data.csv', sep='\t', encoding='utf-8')